In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/CS 839 Project/DeepFill
!pip install tensorflow==1.13.1
!pip install git+https://github.com/JiahuiYu/neuralgym

/content/drive/.shortcut-targets-by-id/1LNbuJz8fbdjBHvbalNzkM3MflzgGgMZv/CS 839 Project/DeepFill
  Cloning https://github.com/JiahuiYu/neuralgym to /tmp/pip-req-build-7n4w28dx
  Running command git clone -q https://github.com/JiahuiYu/neuralgym /tmp/pip-req-build-7n4w28dx


In [ ]:
#Creating mask with Margin
#Based on DeepFill2

import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from PIL import Image
import pickle
import os 

def isSafe(i, j, newarray, vis):
  return i<len(newarray) and j<len(newarray[0]) and i>=0 and j>=0 and vis[i][j]==False


def BFS(array, vis, i , j, temparray, margin):
  queue = []
  queue.append((i,j,0))
  vis[i][j]=True
  while queue:
    x,y,dist = queue.pop(0)
    neighbors = [(x-1,y),(x+1,y),(x-1,y-1),(x+1,y+1),(x-1,y+1),(x+1,y-1),(x,y-1),(x,y+1)]
    for n in neighbors:
        if 0 <= n[0] <= len(array)-1 and 0 <= n[1] <= len(array[0])-1 and array[n[0]][n[1]]==False and vis[n[0]][n[1]]==False and dist<margin:
            vis[n[0]][n[1]]=True
            temparray[n[0]][n[1]]=True
            queue.append((n[0],n[1], dist+1))

def callBFS(array, margin, vis, temparray):
  for i in range(len(array)):
    for j in range(len(array[0])):
      if(array[i][j]==True):
        BFS(array, vis, i, j, temparray, margin)

def CreateMaskDeepFill(actualImagePath,PickelFilePath,BasePath,MarginValue):
  with open(PickelFilePath,'rb') as f: 
    imageArray = pickle.load(f)
  backup_image = np.array(Image.open(actualImagePath))
  backup_image.setflags(write=1)
  num_image = len(imageArray)
  #fig = plt.figure(figsize=(30,num_image))
  for imageid in range(num_image):
    temparray = imageArray[imageid]
    n,m,_ = temparray.shape
    newarray = np.copy(temparray)
    newarray = newarray.reshape(n,m)
    count = np.count_nonzero(newarray == 1)
    vis = np.zeros_like(newarray)
    temparray = np.zeros_like(newarray, dtype=bool)
    #print(count)
    margin = count//MarginValue
    #print(margin)
    callBFS(newarray, margin, vis, temparray)
    newarray = np.bitwise_or(newarray, temparray)
    im = Image.fromarray(newarray)
    im.save(BasePath+"/HeuMask_DeepFill"+str(imageid+1)+".png")
    #ax = fig.add_subplot(1,num_image,imageid+1)
    #ax.imshow(newarray, aspect='auto', cmap=plt.cm.gray, interpolation='nearest')
    #print(newarray)
    image = np.copy(backup_image)
    for i in range(len(image)):
      for j in range(len(image[0])):
        if newarray[i][j] == True:
          image[i][j][0] = 255
          image[i][j][1] = 255
          image[i][j][2] = 255

    im = Image.fromarray(image)
    im.save(BasePath+"/HeuFreeformInputDeepFill"+str(imageid+1)+".png")    
    #ax = fig.add_subplot(1,num_image,imageid+2)
    #ax.imshow(image, aspect='auto')
  
def CreateMaskDeepPrior(PickelFilePath,BasePath,MarginValue):
  with open(PickelFilePath,'rb') as f: 
    imageArray = pickle.load(f)
  num_image = len(imageArray)
  for imageid in range(num_image):
    temparray = imageArray[imageid]
    n,m,_ = temparray.shape
    newarray = np.copy(temparray)
    newarray = newarray.reshape(n,m)
    count = np.count_nonzero(newarray == 1)
    vis = np.zeros_like(newarray)
    temparray = np.zeros_like(newarray, dtype=bool)
    #print(count)
    margin = count//MarginValue
    #print(margin)
    callBFS(newarray, margin, vis, temparray)
    newarray = np.bitwise_or(newarray, temparray)
    newarray = np.invert(newarray)
    im = Image.fromarray(newarray)
    im.save(BasePath+"/Mask"+str(imageid+1)+"_kate.png")
    im.save(BasePath+"/Mask"+str(imageid+1)+"_vase.png")
    im.save(BasePath+"/Mask"+str(imageid+1)+"_library.png")

    





  



In [ ]:
#DeepfillV2
num_actualImages = 10
import os
Base = '/content/drive/MyDrive/CS 839 Project/Data'
OutputBase = '/content/drive/MyDrive/CS 839 Project/DeepFill/generative_inpainting/examples/Heuristic'

# for i in range(1,num_actualImages+1):
for i in range(5,6):  
  imagePath = Base+"/Images/"+str(i)+".jpg"
  pickleDataPath = Base+"/pickleData/"+str(i)+".pkl"
  directory = str(i)
  maskPathDeepFill = os.path.join(OutputBase, directory)

  os.makedirs(maskPathDeepFill,exist_ok=True)

  MarginValue = 200
  CreateMaskDeepFill(imagePath,pickleDataPath,maskPathDeepFill,MarginValue)




In [ ]:
#DeepPrior

#DeepfillV2
num_actualImages = 10
import os
Base = '/content/drive/MyDrive/CS 839 Project/Data'
OutputBaseDeepPrior = '/content/drive/MyDrive/CS 839 Project/DeepPrior'
# for i in range(1,num_actualImages+1):
for i in range(5,6):
  imagePath = Base+"/Images/"+str(i)+".jpg"
  pickleDataPath = Base+"/pickleData/"+str(i)+".pkl"
  directory = str(i)
  maskPathDeepPrior = os.path.join(OutputBaseDeepPrior, directory)
  os.makedirs(maskPathDeepPrior,exist_ok=True)
  MarginValue = 5000

  CreateMaskDeepPrior(pickleDataPath,maskPathDeepPrior,MarginValue)

In [ ]:
%cd /content/drive/MyDrive/CS 839 Project/DeepFill
!pip install tensorflow==1.13.1
!pip install git+https://github.com/JiahuiYu/neuralgym

/content/drive/.shortcut-targets-by-id/1LNbuJz8fbdjBHvbalNzkM3MflzgGgMZv/CS 839 Project/DeepFill
  Cloning https://github.com/JiahuiYu/neuralgym to /tmp/pip-req-build-tzv5nvz7
  Running command git clone -q https://github.com/JiahuiYu/neuralgym /tmp/pip-req-build-tzv5nvz7


In [ ]:
%cd /content/drive/MyDrive/CS 839 Project/DeepFill/generative_inpainting
!python test.py --image examples/Heuristic/5/HeuFreeformInputDeepFill1.png --mask examples/Heuristic/5/HeuMask_DeepFill1.png --output examples/Heuristic/5/Output1.png --checkpoint_dir model_logs/release_places2_256_deepfill_v2

/content/drive/.shortcut-targets-by-id/1LNbuJz8fbdjBHvbalNzkM3MflzgGgMZv/CS 839 Project/DeepFill/generative_inpainting
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [ ]:
%cd /content/drive/MyDrive/CS 839 Project/DeepFill/generative_inpainting
!python test.py --image examples/Heuristic/8/HeuFreeformInputDeepFill1.png --mask examples/Heuristic/8/HeuMask_DeepFill1.png --output examples/Heuristic/8/Output1.png --checkpoint_dir model_logs/release_places2_256_deepfill_v2

/content/drive/.shortcut-targets-by-id/1LNbuJz8fbdjBHvbalNzkM3MflzgGgMZv/CS 839 Project/DeepFill/generative_inpainting
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [ ]:
%cd /content/drive/MyDrive/CS 839 Project/DeepFill/generative_inpainting
!python test.py --image examples/Heuristic/8/HeuFreeformInputDeepFill1.png --mask examples/Heuristic/8/HeuMask_DeepFill1.png --output examples/Heuristic/8/Output1.png --checkpoint_dir model_logs/release_places2_256_deepfill_v2

/content/drive/.shortcut-targets-by-id/1LNbuJz8fbdjBHvbalNzkM3MflzgGgMZv/CS 839 Project/DeepFill/generative_inpainting
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [ ]:
%cd /content/drive/MyDrive/CS 839 Project/DeepFill/generative_inpainting
!python test.py --image examples/Heuristic/1/HeuFreeformInputDeepFill3.png --mask examples/Heuristic/1/HeuMask_DeepFill3.png --output examples/Heuristic/1/Output3.png --checkpoint_dir model_logs/release_places2_256_deepfill_v2

/content/drive/.shortcut-targets-by-id/1LNbuJz8fbdjBHvbalNzkM3MflzgGgMZv/CS 839 Project/DeepFill/generative_inpainting
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])